In [16]:
import json
import gcsfs
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

from utils import smooth_inputs
from cm26_utils import load_and_combine_cm26

# set up gcs filesystem
# 👇 replace with your key 
with open('/home/jovyan/keys/pangeo-forge-ocean-transport-4967-347e2048c5a1.json') as token_file:
    token = json.load(token_file)
fs = gcsfs.GCSFileSystem(token=token)

In [25]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jbusecke/proxy/8787/status,
Dashboard: /user/jbusecke/proxy/8787/status,Workers: 4
Total threads: 16,Total memory: 58.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44839,Workers: 4
Dashboard: /user/jbusecke/proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 58.88 GiB
Comm: tcp://127.0.0.1:37891,Total threads: 4
Dashboard: /user/jbusecke/proxy/44169/status,Memory: 14.72 GiB
Nanny: tcp://127.0.0.1:42117,


In [3]:
fs.ls('ocean-transport-group/scale-aware-air-sea/flux-output')

['ocean-transport-group/scale-aware-air-sea/flux-output/cm26.zarr',
 'ocean-transport-group/scale-aware-air-sea/flux-output/cm26_sample.zarr',
 'ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split.zarr',
 'ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split_full.zarr']

In [18]:
fs.ls('ocean-transport-group/scale-aware-air-sea/preprocessed')

['ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed.zarr',
 'ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_new.zarr',
 'ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_split.zarr',
 'ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_test_split.zarr']

In [4]:
# TODO: Remove old ones once new works

In [5]:
mapper = fs.get_mapper('ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split_full.zarr')

In [12]:
# import zarr
# zarr.consolidate_metadata(mapper)

In [19]:
mapper_smoothed = fs.get_mapper('ocean-transport-group/scale-aware-air-sea/preprocessed/CM26_smoothed_split.zarr')
ds_smoothed = xr.open_dataset(mapper_smoothed, engine='zarr', chunks='auto', consolidated=True)
ds_smoothed

<xarray.Dataset>
Dimensions:                (time: 7305, xt_ocean: 3600, yt_ocean: 2700)
Coordinates:
  * time                   (time) object 0181-01-01 12:00:00 ... 0200-12-31 1...
  * xt_ocean               (xt_ocean) float64 -279.9 -279.8 ... 79.85 79.95
  * yt_ocean               (yt_ocean) float64 -81.11 -81.07 ... 89.94 89.98
Data variables:
    q_ref_smoothed         (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    slp_smoothed           (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    surface_temp_smoothed  (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    t_ref_smoothed         (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    u_relative_smoothed    (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    v_relative_smoothed    (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [13]:
mapper_flux = fs.get_mapper('ocean-transport-group/scale-aware-air-sea/flux-output/cm26_split_full.zarr')
ds_flux = xr.open_dataset(mapper_flux, engine='zarr', chunks='auto', consolidated=True)
# extract the filter scale used (TODO: Check that it is the same for all variables?
filter_scale = ds_flux.ql.attrs['filter_scale']
ds_flux

,Array,Chunk
Bytes,3.10 TiB,37.08 MiB
Shape,"(3, 4, 7305, 3600, 2700)","(1, 1, 1, 3600, 2700)"
Count,2 Graph Layers,87660 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,3.10 TiB,37.08 MiB
Shape,"(3, 4, 7305, 3600, 2700)","(1, 1, 1, 3600, 2700)"
Count,2 Graph Layers,87660 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [17]:
ds_original = load_and_combine_cm26(fs)
ds_original

Load Data
Align in time
Interpolating ocean velocities
Regrid Atmospheric Data
Modify units
Mask nans
Calculate relative wind


<xarray.Dataset>
Dimensions:       (time: 7305, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time          (time) object 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean      (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [22]:
# combine all fields
ds = xr.merge([ds_smoothed, ds_original])
ds

<xarray.Dataset>
Dimensions:                (time: 7305, xt_ocean: 3600, yt_ocean: 2700)
Coordinates:
  * time                   (time) object 0181-01-01 12:00:00 ... 0200-12-31 1...
  * xt_ocean               (xt_ocean) float64 -279.9 -279.8 ... 79.85 79.95
  * yt_ocean               (yt_ocean) float64 -81.11 -81.07 ... 89.94 89.98
Data variables: (12/17)
    q_ref_smoothed         (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    slp_smoothed           (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    surface_temp_smoothed  (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    t_ref_smoothed         (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    u_relative_smoothed    (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    v_relative_smoothed    (time, xt_ocean, yt_ocean) float32 dask.array<chunksize=(3, 3600, 2700), meta=np.ndarray>
    ...                     ...
    wind                   (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp           (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean                (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean                (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative             (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative             (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [26]:
ds_small_scale = (ds_flux.sel(smoothing='smooth_full') - ds_flux).isel(smoothing=slice(1,None))
ds_small_scale

,Array,Chunk
Bytes,2.32 TiB,37.08 MiB
Shape,"(3, 7305, 3600, 2700, 3)","(1, 1, 3600, 2700, 1)"
Count,8 Graph Layers,65745 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,2.32 TiB,37.08 MiB
Shape,"(3, 7305, 3600, 2700, 3)","(1, 1, 3600, 2700, 1)"
Count,8 Graph Layers,65745 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


## 🚨 Temporary patch with the smoothing...
Need to adjust this once I have written out the data without smoothing after the fluxes (this should only be applied on the mean)

In [38]:
def smooth_inputs_dataset(ds:xr.Dataset, dims:list, filter_scale:int, timedim:str='time') -> xr.Dataset:
    """Wrapper that filters a whole dataset, generating a wet_mask from the nanmask of the first timestep (if time is present)."""
    ds_out = xr.Dataset()
    for var in ds.data_vars:
        da = ds[var]
        if timedim in da.dims:
            mask_da = da.isel({timedim:0})
        else:
            mask_da = da
            
        wet_mask = ~np.isnan(mask_da)
        ds_out[var] = smooth_inputs(da, wet_mask, dims, filter_scale)
    return ds_out

In [ ]:
def small_scale_plot(ds_without_time, filter_scale):
    # this should eventually be applied to smooth_all too
    smooth_vars = ['smooth_vel', 'smooth_tracer']

    datasets = []
    for sm in ds_small_scale_mean.smoothing:
        ds_partial = ds_small_scale_mean.sel(smoothing=sm)
        if sm in smooth_vars:
            datasets.append(smooth_inputs_dataset(ds_partial, ['xt_ocean', 'yt_ocean'], filter_scale))
        else:
            datasets.append(ds_partial)

    ds_plot = xr.concat(datasets, dim='smoothing')
    

In [41]:
ds_small_scale_mean = ds_small_scale.mean('time').load()
ds_small_scale_mean

2022-10-28 19:52:36,235 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x7fa65dbda5e0>>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/bokeh/server/tornado.py", line 760, in _keep_alive
    c.send_ping()
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/bokeh/server/connection.py", line 93, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/tornado/websocket.py", line 445, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
2022-10-28 19:52:37,248 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x7fa65dbda5e0>

<xarray.Dataset>
Dimensions:    (algo: 3, smoothing: 3, xt_ocean: 3600, yt_ocean: 2700)
Coordinates:
  * algo       (algo) <U8 'ncar' 'ecmwf' 'coare3p0'
  * smoothing  (smoothing) <U13 'smooth_tracer' 'smooth_vel' 'smooth_all'
  * xt_ocean   (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean   (yt_ocean) float64 -81.11 -81.07 -81.02 ... 89.89 89.94 89.98
Data variables:
    evap       (smoothing, algo, xt_ocean, yt_ocean) float32 dask.array<chunksize=(1, 1, 3600, 2700), meta=np.ndarray>
    qh         (smoothing, algo, xt_ocean, yt_ocean) float32 dask.array<chunksize=(1, 1, 3600, 2700), meta=np.ndarray>
    ql         (smoothing, algo, xt_ocean, yt_ocean) float32 dask.array<chunksize=(1, 1, 3600, 2700), meta=np.ndarray>
    taux       (smoothing, algo, xt_ocean, yt_ocean) float32 dask.array<chunksize=(1, 1, 3600, 2700), meta=np.ndarray>
    tauy       (smoothing, algo, xt_ocean, yt_ocean) float32 dask.array<chunksize=(1, 1, 3600, 2700), meta=np.ndarray>

In [40]:
ds_small_scale_mean = ds_small_scale_mean.load()

2022-10-28 19:49:50,375 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-10-28 19:49:50,375 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-10-28 19:49:50,374 - distributed.worker - WARNING - Heartbeat to scheduler failed
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/worker.py", line 1191, in heartbeat
    response = await retry_operation(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/utils_comm.py", line 383, in retry_operation
    return await retry(
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/utils_comm.py", line 368